In [58]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten

In [93]:
data=pd.read_csv('WhoScoredFinal1.csv', delimiter=',')

dataDefender=data[(data['positionText']=='Defender')][['playerId','lastName','tacklePerGame','interceptionPerGame','clearancePerGame','rating']]
dataDefenderFeatures=dataDefender.groupby('playerId')['tacklePerGame','interceptionPerGame','clearancePerGame'].mean().fillna(0)
dataDefenderLables=dataDefender.groupby('playerId')['rating'].mean().fillna(0

C:\Users\Dantez\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
<ipython-input-93-0a398895d0cb>:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  dataDefenderFeatures=dataDefender.groupby('playerId')['tacklePerGame','interceptionPerGame','clearancePerGame'].mean().fillna(0)


In [69]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing


trainFeatures, testFeatures, trainLables, testLabels = train_test_split(dataDefenderFeatures, dataDefenderLables, test_size=0.25)

min_max_scaler = preprocessing.MinMaxScaler()
trainFeaturesScaled = pd.DataFrame(min_max_scaler.fit_transform(trainFeatures))
testFeaturesScaled = pd.DataFrame(min_max_scaler.fit_transform(testFeatures))

trainLablesScaled = pd.DataFrame(min_max_scaler.fit_transform(trainLables.to_numpy().reshape(-1, 1)))
testLabelsScaled = pd.DataFrame(min_max_scaler.fit_transform(testLabels.to_numpy().reshape(-1, 1)))

In [88]:
testLabels

playerId
334438    6.127500
94906     6.710727
110419    6.581667
384391    6.565000
131875    6.550000
            ...   
239259    6.896042
146799    6.346667
1354      6.693105
89830     6.595333
369553    6.534500
Name: rating, Length: 1895, dtype: float64

In [71]:
# MLP

modelMLP = Sequential()
modelMLP.add(Dense(3, input_shape=(3, ), activation='relu'))
modelMLP.add(Dense(3, activation='relu'))
modelMLP.add(Dense(1, activation='relu'))
modelMLP.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 3)                 12        
_________________________________________________________________
dense_22 (Dense)             (None, 3)                 12        
_________________________________________________________________
dense_23 (Dense)             (None, 1)                 4         
Total params: 28
Trainable params: 28
Non-trainable params: 0
_________________________________________________________________


In [114]:
modelMLP.compile(loss='mean_squared_error',
              optimizer= 'adam',
              metrics=['mean_squared_error'])

historyMLP = modelMLP.fit(trainFeaturesScaled, trainLables, epochs=100, batch_size=1000,
                    validation_data=(testFeaturesScaled, testLabelsScaled))

Epoch 1/100
6/6 [==============================] - 1s 43ms/step - loss: 36.8429 - mean_squared_error: 36.8429 - val_loss: 36.3117 - val_mean_squared_error: 36.3117
Epoch 2/100
6/6 [==============================] - 0s 9ms/step - loss: 36.5046 - mean_squared_error: 36.5046 - val_loss: 35.9712 - val_mean_squared_error: 35.9712
Epoch 3/100
6/6 [==============================] - 0s 9ms/step - loss: 36.1896 - mean_squared_error: 36.1896 - val_loss: 35.6244 - val_mean_squared_error: 35.6244
Epoch 4/100
6/6 [==============================] - 0s 10ms/step - loss: 35.8456 - mean_squared_error: 35.8456 - val_loss: 35.2711 - val_mean_squared_error: 35.2711
Epoch 5/100
6/6 [==============================] - 0s 9ms/step - loss: 35.4457 - mean_squared_error: 35.4457 - val_loss: 34.9111 - val_mean_squared_error: 34.9111
Epoch 6/100
6/6 [==============================] - 0s 10ms/step - loss: 35.1134 - mean_squared_error: 35.1134 - val_loss: 34.5434 - val_mean_squared_error: 34.5434
Epoch 7/100
6/6 [==

Epoch 51/100
6/6 [==============================] - 0s 9ms/step - loss: 12.7814 - mean_squared_error: 12.7814 - val_loss: 11.5903 - val_mean_squared_error: 11.5903
Epoch 52/100
6/6 [==============================] - 0s 14ms/step - loss: 12.2602 - mean_squared_error: 12.2602 - val_loss: 11.0822 - val_mean_squared_error: 11.0822
Epoch 53/100
6/6 [==============================] - 0s 10ms/step - loss: 11.7857 - mean_squared_error: 11.7857 - val_loss: 10.5850 - val_mean_squared_error: 10.5850
Epoch 54/100
6/6 [==============================] - 0s 10ms/step - loss: 11.2209 - mean_squared_error: 11.2209 - val_loss: 10.0988 - val_mean_squared_error: 10.0988
Epoch 55/100
6/6 [==============================] - 0s 16ms/step - loss: 10.7703 - mean_squared_error: 10.7703 - val_loss: 9.6249 - val_mean_squared_error: 9.6249
Epoch 56/100
6/6 [==============================] - 0s 10ms/step - loss: 10.2604 - mean_squared_error: 10.2604 - val_loss: 9.1638 - val_mean_squared_error: 9.1638
Epoch 57/100
6/

In [140]:
predicted = modelMLP.predict(testFeaturesScaled)
# predicted
indexes = np.array(testLabels.index)
indexes = indexes.reshape(1895 ,1)
labels = np.array(testLabels).reshape(1895 ,1)
np.concatenate((indexes,labels,predicted), axis=1)

array([[3.34438000e+05, 6.12750000e+00, 4.51241207e+00],
       [9.49060000e+04, 6.71072657e+00, 7.11121607e+00],
       [1.10419000e+05, 6.58166667e+00, 7.79043198e+00],
       ...,
       [1.35400000e+03, 6.69310526e+00, 7.39607525e+00],
       [8.98300000e+04, 6.59533333e+00, 6.41636038e+00],
       [3.69553000e+05, 6.53450000e+00, 7.09276342e+00]])